In [2]:
from datasets import load_dataset

ds = load_dataset("graphs-datasets/MUTAG")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

full.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/188 [00:00<?, ? examples/s]

In [208]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
nx.random.seed(SEED)

In [195]:
import random
import numpy as np
import networkx as nx

def sample_pos_neg_from_graph(adj_mat,
                              num_pos_samples=3,
                              num_neg_samples=3,
                              k_hop=3,
                              max_neg_attempts=100,
                              max_subgraph_size=None):
    """
    For a single graph adjacency matrix `adj_mat` (numpy array),
    returns lists of positive subgraph adjacency matrices and negative subgraph adjacency matrices,
    and the node lists for the positive samples.

    - max_subgraph_size: int or None. If int, limit the number of nodes in each positive subgraph.
    """
    # number of nodes in full graph
    n = adj_mat.shape[0]

    # build full networkx graph for negative-isomorphism checks
    G = nx.from_numpy_array(adj_mat)

    pos_adj_mats = []
    pos_nodes = []  # each entry is a list of original node indices in that positive sample

    # create positive k-hop induced subgraphs (with optional size cap)
    for _ in range(num_pos_samples):
        seed = random.randrange(n)
        sub_nodes = set([seed])
        frontier = set([seed])

        for hop in range(k_hop):
            new_nodes = set()
            # gather neighbors of current frontier
            frontier_list = list(frontier)
            random.shuffle(list(frontier_list))
            for node in frontier_list:
                neighs = set(np.where(adj_mat[node])[0].tolist())
                # only consider neighbors not already in sub_nodes
                new_nodes.update(neighs - sub_nodes)

            if not new_nodes:
                break  # no more nodes to add

            # if adding all new_nodes would exceed max_subgraph_size, sample a subset
            if max_subgraph_size is not None:
                remaining_capacity = max_subgraph_size - len(sub_nodes)
                if remaining_capacity <= 0:
                    break
                if len(new_nodes) > remaining_capacity:
                    new_nodes = set(random.sample(list(new_nodes), remaining_capacity))


            # update sub_nodes and set new frontier to the nodes we just added
            sub_nodes.update(new_nodes)
            frontier = new_nodes

        # final node list (sorted for stable indexing)
        node_list = sorted(sub_nodes)
        if len(node_list) == 0:
            node_list = [seed]

        # if after everything the node_list still exceeds max_subgraph_size (defensive),
        # randomly keep max_subgraph_size nodes and recompute induced subgraph.
        if max_subgraph_size is not None and len(node_list) > max_subgraph_size:
            node_list = sorted(random.sample(node_list, max_subgraph_size))

        # extract induced adjacency using np.ix_
        sub_adj = adj_mat[np.ix_(node_list, node_list)].copy()
        pos_adj_mats.append(sub_adj)
        pos_nodes.append(node_list)

    neg_adj_mats = []

    # create negative samples by *adding* a non-edge inside the positive subgraph
    # and verifying that the resulting small graph is NOT present as a subgraph in the full graph
    # (i.e., we want the modified subgraph to be non-isomorphic to any subgraph of G)
    if len(pos_adj_mats) == 0:
        return pos_adj_mats, pos_nodes, neg_adj_mats, adj_mat

    for k in range(num_neg_samples):
        pos_idx = k % len(pos_adj_mats)
        sub_adj = pos_adj_mats[pos_idx]
        nodes_in_sub = pos_nodes[pos_idx]
        m = sub_adj.shape[0]

        # list all absent undirected pairs in the upper triangle (i < j) where there is no edge
        absent_pairs = [(i, j) for i in range(m) for j in range(i+1, m) if sub_adj[i, j] == 0]

        if not absent_pairs:
            # no possible "add-edge" negative samples inside this subgraph
            continue

        # shuffle candidate absent pairs to randomize attempts
        random.shuffle(absent_pairs)

        attempts = 0
        found_neg = False
        # iterate candidates but stop after max_neg_attempts total tries
        for (i, j) in absent_pairs:
            if attempts >= max_neg_attempts:
                break
            attempts += 1

            neg_adj = sub_adj.copy()
            neg_adj[i, j] = 1
            neg_adj[j, i] = 1

            # Build small graph H from this adjacency (nodes are 0..m-1)
            H = nx.from_numpy_array(neg_adj)

            # Check if H is subgraph-isomorphic to the full graph G
            iso = nx.isomorphism.GraphMatcher(G, H)
            if not iso.subgraph_is_isomorphic():
                # successful negative sample: store (neg_adj, nodes_in_sub) so mapping is known
                neg_adj_mats.append((neg_adj))
                found_neg = True
                break

        # if we didn't find a negative sample in the deterministic pass above, try random sampling
        if not found_neg and attempts < max_neg_attempts:
            # try random pairs (with replacement) up to remaining attempts
            remaining = max_neg_attempts - attempts
            for _ in range(remaining):
                i, j = random.choice(absent_pairs)
                neg_adj = sub_adj.copy()
                neg_adj[i, j] = 1
                neg_adj[j, i] = 1

                H = nx.from_numpy_array(neg_adj)
                iso = nx.isomorphism.GraphMatcher(G, H)
                if not iso.subgraph_is_isomorphic():
                    neg_adj_mats.append((neg_adj))
                    found_neg = True
                    break

        # if still not found, we skip this negative sample (no valid negative within attempts)

    return pos_adj_mats, pos_nodes, neg_adj_mats, adj_mat


In [196]:
# Example loop over dataset
num_pos_samples = 3
num_neg_samples = 3
k_hop = 3

mutag_dataset_dict = {}
for i, graph in tqdm(enumerate(ds['train'])):
    n = graph['num_nodes']
    mutag_dataset_dict[i] = {}
    adj_mat = np.zeros((n, n), dtype=np.uint8)
    srcs = graph['edge_index'][0]
    dsts = graph['edge_index'][1]
    for u, v in zip(srcs, dsts):
        adj_mat[u, v] = 1
        adj_mat[v, u] = 1

    pos_adj_mats, pos_nodes, neg_adj_mats, adj_mat = sample_pos_neg_from_graph(
        adj_mat,
        num_pos_samples=num_pos_samples,
        num_neg_samples=num_neg_samples,
        k_hop=3,
        max_subgraph_size = 10,
        max_neg_attempts=1000
    )
    mutag_dataset_dict[i]["graph_adj_mat"] = adj_mat
    mutag_dataset_dict[i]["positive_subgraph_adj_mats"] = pos_adj_mats
    mutag_dataset_dict[i]["negative_subgraph_adj_mats"] = neg_adj_mats
    for key in graph.keys():
      mutag_dataset_dict[i][key] = graph[key]

188it [00:01, 146.04it/s]


In [198]:
shapelist = []
for graph in mutag_dataset_dict.values():
  shapelist.append(graph["negative_subgraph_adj_mats"][0].shape[0])
shapelist = np.array(shapelist)
shapelist.min(), shapelist.max(), shapelist.mean()

(np.int64(5), np.int64(10), np.float64(8.686170212765957))

In [206]:
with open('/congress.edgelist', 'r') as file:
    snap_ds = file.readlines()
raw_snap_dict = {'source': [], 'dest': [], 'weight':[]}

for i, line in enumerate(snap_ds):
  element_list = line.split()
  raw_snap_dict['source'].append(int(element_list[0]))
  raw_snap_dict['dest'].append(int(element_list[1]))
  raw_snap_dict['weight'].append(float(element_list[3][:-1]))

n = len(set(raw_snap_dict['source']).union(set(raw_snap_dict['dest'])))
adj_mat = np.zeros((n, n), dtype=np.uint8)

for u, v in zip(raw_snap_dict['source'], raw_snap_dict['dest']):
    adj_mat[u, v] = 1
    adj_mat[v, u] = 1
sampled_graph_adj_mats, _, _, _ = sample_pos_neg_from_graph(
      adj_mat,
      num_pos_samples=50,
      num_neg_samples=0,
      k_hop=3,
      max_neg_attempts=0,
      max_subgraph_size = 30
  )


In [207]:
snap_dataset_dict = {}

for i, graph in tqdm(enumerate(sampled_graph_adj_mats)):
    snap_dataset_dict[i] = {}
    n = graph.shape[0]

    pos_adj_mats, _, neg_adj_mats, _ = sample_pos_neg_from_graph(
        graph,
        num_pos_samples=3,
        num_neg_samples=3,
        k_hop=3,
        max_subgraph_size = 10,
        max_neg_attempts=10)
    snap_dataset_dict[i]["graph_adj_mat"] = graph
    snap_dataset_dict[i]["positive_subgraph_adj_mats"] = pos_adj_mats
    snap_dataset_dict[i]["negative_subgraph_adj_mats"] = neg_adj_mats

50it [29:31, 35.43s/it]


In [210]:


MASTER_SEED = 42
master_rng = random.Random(MASTER_SEED)

def er_parent(n=30, p=0.1, seed=None, require_connected=True, stitch=False):
    g = nx.erdos_renyi_graph(n, p, seed=seed)
    if require_connected:
        if stitch:
            components = list(nx.connected_components(g))
            if len(components) > 1:
                reps = [random.choice(list(c)) for c in components]
                for i in range(len(reps)-1):
                    g.add_edge(reps[i], reps[i+1])
        else:
            # resample until connected (may bias distribution slightly)
            while not nx.is_connected(g):
                g = nx.erdos_renyi_graph(n, p, seed=master_rng.randint(0, 2**31-1))
    return g

def adj_from_nx(g):
    return np.array(nx.to_numpy_array(g)).astype(np.uint8)

er_dataset_dict, ba_dataset_dict = {}, {}

NUM = 100
for i in tqdm(range(NUM)):
    s_er = master_rng.randint(0, 2**31-1)
    s_ba = master_rng.randint(0, 2**31-1)

    er_g = er_parent(n=30, p=0.1 + master_rng.random()/30.0, seed=s_er, require_connected=True, stitch=True)
    ba_g = nx.barabasi_albert_graph(30, master_rng.choice([1,2,3,5]), seed=s_ba)

    er_adj_mat = adj_from_nx(er_g)
    ba_adj_mat = adj_from_nx(ba_g)

    # sample...
    pos_adj_mats, _, neg_adj_mats, _ = sample_pos_neg_from_graph(
        er_adj_mat, num_pos_samples=3, num_neg_samples=3,
        k_hop=3, max_subgraph_size=10, max_neg_attempts=100
    )
    er_dataset_dict[i] = {
        "graph_adj_mat": er_adj_mat,
        "positive_subgraph_adj_mats": pos_adj_mats,
        "negative_subgraph_adj_mats": neg_adj_mats
    }

    pos_adj_mats, _, neg_adj_mats, _ = sample_pos_neg_from_graph(
        ba_adj_mat, num_pos_samples=3, num_neg_samples=3,
        k_hop=3, max_subgraph_size=10, max_neg_attempts=100
    )
    ba_dataset_dict[i] = {
        "graph_adj_mat": ba_adj_mat,
        "positive_subgraph_adj_mats": pos_adj_mats,
        "negative_subgraph_adj_mats": neg_adj_mats
    }


100%|██████████| 100/100 [07:57<00:00,  4.77s/it]


In [214]:
import pickle, gzip
with gzip.open("/mutag_benchmark.pkl", "wb") as f:
    pickle.dump(mutag_dataset_dict, f)
with gzip.open("/snap_benchmark.pkl", "wb") as f:
    pickle.dump(snap_dataset_dict, f)
with gzip.open("/er_benchmark.pkl", "wb") as f:
    pickle.dump(er_dataset_dict, f)
with gzip.open("/ba_benchmark.pkl", "wb") as f:
    pickle.dump(ba_dataset_dict, f)